<a href="https://colab.research.google.com/github/vyshanviG/Customer-segment-insights/blob/main/Untitled10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
from google.colab import files
uploaded = files.upload()

Saving customers.csv.csv to customers.csv.csv


In [10]:
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
orders = pd.read_csv("orders.csv.csv")
items = pd.read_csv("order_items.csv.csv")
payments = pd.read_csv("order_payments.csv.csv")
customers = pd.read_csv("customers.csv.csv")
orders = orders[orders['order_status'] == 'delivered']
orders['order_purchase_timestamp'] = pd.to_datetime(orders['order_purchase_timestamp'])
# Merge orders with customers
df = pd.merge(orders, customers, on='customer_id', how='inner')

# Merge with payments
df = pd.merge(df, payments[['order_id', 'payment_value']], on='order_id', how='left')
# Set reference date
snapshot_date = df['order_purchase_timestamp'].max() + pd.Timedelta(days=1)

rfm = df.groupby('customer_id').agg({
    'order_purchase_timestamp': lambda x: (snapshot_date - x.max()).days,
    'order_id': 'nunique',
    'payment_value': 'sum'
}).reset_index()

rfm.columns = ['customer_id', 'Recency', 'Frequency', 'Monetary']
rfm['R'] = pd.qcut(rfm['Recency'], 4, labels=[4,3,2,1])
rfm['F'] = pd.qcut(rfm['Frequency'].rank(method='first'), 4, labels=[1,2,3,4])
rfm['M'] = pd.qcut(rfm['Monetary'], 4, labels=[1,2,3,4])
rfm['RFM_Score'] = rfm[['R','F','M']].sum(axis=1).astype(int)
def segment_customer(score):
    if score >= 9:
        return 'VIP'
    elif score >= 7:
        return 'Loyal'
    elif score >= 5:
        return 'Potential'
    else:
        return 'At Risk'

rfm['Segment'] = rfm['RFM_Score'].apply(segment_customer)
rfm.to_csv('final_segments.csv', index=False)
rfm


/tmp/ipython-input-3333542906.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  orders['order_purchase_timestamp'] = pd.to_datetime(orders['order_purchase_timestamp'])


,customer_id,Recency,Frequency,Monetary,R,F,M,RFM_Score,Segment
0,00012a2ce6f8dcda20d059ce98491703,288,1,114.74,2,1,3,6,Potential
1,000161a058600d5901f007fab4c27140,410,1,67.41,1,1,2,4,At Risk
2,0001fd6190edaaf884bcaf3d49edf079,548,1,195.42,1,1,4,6,Potential
3,0002414f95344307404f0ace7a26f1d5,379,1,179.35,1,1,4,6,Potential
4,000379cdec625522490c315e70c7a9fb,150,1,107.01,3,1,3,7,Loyal
...,...,...,...,...,...,...,...,...,...
96473,fffcb937e9dd47a13f05ecb8290f4d3e,166,1,91.91,3,4,2,9,VIP
96474,fffecc9f79fd8c764f843e9951b11341,153,1,81.36,3,4,2,9,VIP
96475,fffeda5b6d849fbd39689bb92087f431,100,1,63.13,4,4,2,10,VIP
96476,ffff42319e9b2d713724ae527742af25,77,1,214.13,4,4,4,12,VIP


In [11]:
import pandas as pd

# Load the saved RFM data
rfm = pd.read_csv('final_segments.csv')  # ← adjust path if needed
# Filter out VIP customers
vip_customers = rfm[rfm['Segment'] == 'VIP']

# Display top 5 VIPs
vip_customers.head(10)

,customer_id,Recency,Frequency,Monetary,R,F,M,RFM_Score,Segment
9,000598caf2ef4117407665ac33275130,19,1,1255.71,4,1,4,9,VIP
44,00205ad9ba1ef4340cef86583294cf82,109,1,928.70,4,1,4,9,VIP
48,002554bdf9eb99618d8189c3a89fdd52,16,1,244.08,4,1,4,9,VIP
58,002ce108ccf0356ef5c8b1dce3c0ae29,84,1,185.60,4,1,4,9,VIP
62,002ef7e55600d44ead53f7c1644e5222,71,1,199.02,4,1,4,9,VIP
93,0040a8417928d0d5abd5169cd7877181,63,1,219.49,4,1,4,9,VIP
101,00447b6bd39c4a0f6366b05948f9e2e3,65,1,303.79,4,1,4,9,VIP
136,0061937e85c708a41f5d5b24405d0f50,66,1,194.04,4,1,4,9,VIP
140,0063913c2f1878cc471de963dc0571c4,20,1,254.58,4,1,4,9,VIP
144,00650b839d32a071b1025af8ca26dd13,40,1,367.11,4,1,4,9,VIP


In [12]:
# Filter At Risk customers
at_risk = rfm[rfm['Segment'] == 'At Risk']

# Show top 5 At Risk customers
at_risk.head()

# Count of At Risk customers
print("Customers At Risk of Churn:", len(at_risk))

Customers At Risk of Churn: 6127


In [13]:
# Group by segment to get averages
rfm.groupby('Segment')[['Recency', 'Frequency', 'Monetary']].mean().sort_values('Frequency', ascending=False)

,Recency,Frequency,Monetary
Segment,,,
At Risk,411.026277,1.0,53.077478
Loyal,237.372252,1.0,150.578544
Potential,321.475103,1.0,93.501588
VIP,144.088714,1.0,245.108558


In [14]:
import pandas as pd
orders = pd.read_csv("orders.csv.csv")
order_items = pd.read_csv("order_items.csv.csv")
payments = pd.read_csv("order_payments.csv.csv")
customers = pd.read_csv("customers.csv.csv")
customers.info()  # check nulls and types
customers.drop_duplicates(inplace=True)
customers.isnull().sum()
orders['order_purchase_timestamp'] = pd.to_datetime(orders['order_purchase_timestamp'])
orders = orders[orders['order_status'] == 'delivered']  # Only keep delivered orders
orders.dropna(subset=['order_purchase_timestamp'], inplace=True)
order_items.isnull().sum()
order_items.dropna(inplace=True)
payments.isnull().sum()
payments.dropna(inplace=True)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 5 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   customer_id               99441 non-null  object
 1   customer_unique_id        99441 non-null  object
 2   customer_zip_code_prefix  99441 non-null  int64 
 3   customer_city             99441 non-null  object
 4   customer_state            99441 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.8+ MB


/tmp/ipython-input-1522126258.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  orders.dropna(subset=['order_purchase_timestamp'], inplace=True)
